In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from scipy import stats
import chart_studio.plotly as py
import plotly.express as px
import cufflinks as cf
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [ ]:
# étape 1 : Nettoyage de la base de données pour retirer les titres qui n'ont pas de startYear
actors_occurence_df8 = actors_occurence_df7[actors_occurence_df7['startYear'] != "\\N"]
# Garder les colonnes primaryName et startYear pour alléger le dataset
actors_occurence_df9 = actors_occurence_df8[["primaryName", "startYear"]]
# Reset de l'index
actors_occurence_df10 = actors_occurence_df9.reset_index(drop=True)

In [ ]:
# étape 2 : création d'une fonction pour transformer une date en décennie
def find_decade(year):
    decade1 = (year // 10 * 10)
    decade2 = (year // 10 * 10) + 10
    return str(decade1) + " - " + str(decade2)

print(find_decade(1934))


In [ ]:
# étape 3 : application de la fonction
actors_occurence_df11 = actors_occurence_df10.astype({"startYear": int})
actors_occurence_df11["startYear"] = actors_occurence_df11["startYear"].apply(find_decade)

In [ ]:
# étape 4 : reset de l'index
actors_occurence_df11.reset_index(inplace=True)

In [ ]:
# étape 5 : groupby sur la startYear et primaryName
df_grouped_notreset  = pd.DataFrame({'count' : actors_occurence_df11.groupby(['startYear', 'primaryName'] ).size()})
df_grouped_notreset.head()

In [ ]:
# étape 6 : placer les résultats du count du plus haut au plus bas
df_grouped_notreset_sorted=df_grouped_notreset.sort_values(['startYear', 'count'], ascending=False)
df_grouped_notreset_sorted.head()

In [ ]:
groupedDf = actors_occurence_df11.groupby(['startYear', 'primaryName'] ).size()
groupedDf.head()

In [ ]:
# étape 7 : Récupérer uniquement les 5 plus hautes valeur de la colonne count
df_final  = pd.DataFrame({'count' : groupedDf.groupby(level='startYear').nlargest(5).reset_index(level=0, drop=True)})

In [ ]:
df_final.reset_index(inplace=True)
df_final["test"] = df_final.index

In [ ]:
df_final2 = df_final.tail(60)

In [ ]:
# # Etape 8 : Sauvegarde du dataframe
df_final2.to_csv(r'C:\Users\Berenger\Desktop\projet abc\acteur_par_periode.csv', index = False, header = True)

In [ ]:
# # Etape 9 : Chargement du dataframe
acteur_par_periode = pd.read_csv(r"C:\Users\Berenger\Desktop\projet abc\acteur_par_periode.csv")
acteur_par_periode.head()

In [ ]:
acteur_par_periode['rank'] = acteur_par_periode.groupby('startYear')['count'].rank(method = 'first')
acteur_par_periode.style.background_gradient(subset=pd.IndexSlice[:, ['rank']])
acteur_par_periode.head(5)

In [ ]:
fig = px.bar(acteur_par_periode, x = 'count', y="rank", text ='primaryName', color = 'primaryName',
    title = 'Quels sont les acteurs les plus présents par périodes ?',
    labels = {'startYear': 'Période', 'primaryName': 'Acteurs'},
    orientation='h',
    animation_frame="startYear",
    range_x=[0,150],
    range_y=[0,6],
    width=1300, height=800)
 
fig.update_traces(textfont_size=12, textposition='outside')
fig.update_layout()
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000

fig.update_layout(showlegend=False, title_x=0.5)